In [ ]:
#@title Imports.
import io
import math
import requests
from typing import List, Optional, Sequence, Tuple
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats

warnings.simplefilter('ignore', category=RuntimeWarning)
from IPython.display import clear_output

In [ ]:
CATCH = 3e7
NUMBER_OF_TIMEBINS = int(1.5e9/CATCH)
LEGEND = ['8:0|8:0', '8:0|7:1', '8:0|6:2', '8:0|5:3', '8:0|4:4']
YLIM = (0, 1)
DOWNSAMPLE = False

In [ ]:
#@title Plot metric function
def plot_metric_style(
    *,
    dataframes: Sequence[pd.DataFrame],
    eval_name: str,
    first_choice: str,
    second_choice: str,
    number_of_timebins: int,
    number_of_agents_to_eval: int,
    titles: List[str],
) -> Tuple[
    np.ndarray,
    List[float],
    List[float],
    Tuple[List[float], ...],
    List[Tuple[np.ndarray, np.ndarray]]]:
  """Plot the bias evaluation for each run.

  Args:
    dataframes: Dataframes that constitute one experiment (one run per plot).
    eval_name: Name of evaluation that was run.
    first_choice: Identity of the first target in the dual choice eval.
    second_choice: Identity of the second target in the dual choice eval.
    number_of_timebins: Number of bins for the x axis.
    number_of_agents_to_eval: Number of agents to evaluate.
    titles: Titles of runs.

  Returns:
    (Nested) tuple of aggregated statistics about the experiment.

  """
  cols = len(dataframes)
  plt.subplots(1, cols, figsize=(cols * 3.5, 3), facecolor='white')
  y_differences = []
  y_errors = []
  y_reds = []
  y_blues = []
  mean_red_errors = []
  mean_blue_errors = []
  raw_ys = []
  for c, df in enumerate(dataframes):

    plt.subplot(1, cols, c + 1)
    x_red, y_red, y_red_error, red_players = get_series(
        df=df,
        player_color='red',
        eval_name=eval_name,
        number_of_agents_to_eval=number_of_agents_to_eval,
        number_of_timebins=number_of_timebins,
    )
    plt.plot(x_red, y_red, 'or')
    plt.plot(x_red, y_red, '-', color='red')
    plt.fill_between(
        x_red, y_red - y_red_error, y_red + y_red_error, color='red', alpha=0.2
    )

    x_blue, y_blue, y_blue_error, blue_players = get_series(
        df=df,
        player_color='blue',
        eval_name=eval_name,
        number_of_agents_to_eval=number_of_agents_to_eval,
        number_of_timebins=number_of_timebins,
    )
    plt.plot(x_blue, y_blue, 'ob')
    plt.plot(x_blue, y_blue, '-', color='blue')
    plt.fill_between(
        x_blue,
        y_blue - y_blue_error,
        y_blue + y_blue_error,
        color='blue',
        alpha=0.2,
    )

    plt.hlines(0.5, 0, np.max(x_red), 'black')
    plt.ylim(0, 1)
    plt.ylabel(f'{second_choice} bias to {first_choice} bias', labelpad=-2)
    plt.xlim(0, 1.5e9)
    plt.xticks([0, 0.5e9, 1e9, 1.5e9], ['0', '0.5e9', '1e9', '1.5e9'])
    plt.xlabel('Training steps')
    plt.title(titles[c])
    assert len(y_blue) == len(y_red)
    c = c + 1

    y_differences.append(y_red - y_blue)
    y_errors.append((y_red_error + y_blue_error) / 2)
    y_reds.append(np.mean(y_red))
    y_blues.append(np.mean(y_blue))
    mean_red_errors.append(np.mean(y_red_error))
    mean_blue_errors.append(np.mean(y_blue_error))
    raw_ys.append((red_players, blue_players))

  discrimination_information = (
      y_reds,
      y_blues,
      mean_red_errors,
      mean_blue_errors,
  )

  plt.savefig('evaluation_metrics.svg', dpi=100)
  return x_red, y_differences, y_errors, discrimination_information, raw_ys


In [ ]:
#@title Load dataframes function
def load_dataframes(*,
                    file_label: str,
                    number_of_files: int,
                    downsample: bool) -> List[pd.DataFrame]:
  """Load dataframes from cloud bucket.

  Args:
    file_label: Label of the filenames.
    number_of_files: How many files are in the experiment (one per run).
    downsample: Whether to downsample or not.

  Returns:
    List of dataframes.

  """
  df_list_output = []
  for file_number in range(number_of_files):
    file_location = (
        f'https://storage.googleapis.com/tabula_rasa_agents/{file_label}_{file_number}.csv'
    )
    response = requests.get(file_location)
    with io.BytesIO(response.content) as f:
      df = pd.read_csv(f, na_values=['NaN']).fillna(0)
      if downsample:
        # The df size is already curtailed to 100k, for ease of use it is
        # reduced even further here.
        df = df.sample(frac=0.5)
      df_list_output.append(df)
  clear_output()
  return df_list_output

In [ ]:
#@title Get data series function
def get_series(
    *,
    df: pd.DataFrame,
    player_color: str,
    eval_name: str,
    number_of_agents_to_eval: int,
    number_of_timebins: int = NUMBER_OF_TIMEBINS,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
  """Obtain data of one eval run.

  Args:
    df: Dataframe of one run.
    player_color: Color of agent that is evaluated.
    eval_name: Name of the evaluation.
    number_of_agents_to_eval:  Number of agents to evaluate.
    number_of_timebins: Number of timebins for the x axis.

  Returns:
    Tuple of arrays containing run eval data.
  """
  reference_x = None
  y_raw = np.zeros((number_of_agents_to_eval, number_of_timebins))
  # The first few agents are the dummies we use to eval.
  for i, p in enumerate(range(8 - number_of_agents_to_eval, 8)):
    # Target 2 is red 3 is blue
    first_target_x, first_target_y = get_player_interactions(
        df=df,
        player_color=player_color,
        player_number=p,
        target_number=2,
        eval_name=eval_name,
    )
    second_target_x, second_target_y = get_player_interactions(
        df=df,
        player_color=player_color,
        player_number=p,
        target_number=3,
        eval_name=eval_name,
    )
    x, y, reference_x = interaction_fraction_within_player(
        first_target_x=first_target_x,
        first_target_y=first_target_y,
        second_target_x=second_target_x,
        second_target_y=second_target_y,
        reference_x=reference_x,
    )
    x = x[0:number_of_timebins]
    y = y[0:number_of_timebins]
    y_raw[i, :] = y
  y_raw = y_raw.T
  y_error = (np.std(y_raw, axis=1) / math.sqrt(y_raw.shape[1])) * 1.96
  y_mean = np.mean(y_raw, axis=1)

  assert x.ndim == 1
  assert y_mean.ndim == 1
  assert y_error.ndim == 1
  return x, y_mean, y_error, y_raw

In [ ]:
#@title Get player interaction function
def get_player_interactions(
    *,
    df: pd.DataFrame,
    player_color: str,
    player_number: int,
    target_number: int,
    eval_name: str,
) -> Tuple[np.ndarray, np.ndarray]:
  """Extract interactions frome evaluation dataframe.

  Args:
    df: Dataframe with evaluation data.
    player_color: Color of the agent to be evaluated, blue or red for
    our purposes.
    player_number: Number of the agent to be evaluated.
    target_number: Number of the target agent.
    eval_name: Name of the evaluation.

  Returns:
    Tuple of numpy arrays with the x (learner steps) and y values
    of the interactions.
  """
  leaner_frames = df[
      f'{player_color}_{player_number}_behavior_{eval_name}:num_learner_frames'
  ].to_numpy()
  interaction_values = df[
      f'{player_color}_{player_number}_behavior_{eval_name}:interaction_player_{target_number}'
  ].to_numpy()
  x = leaner_frames[interaction_values != np.array(None)]
  y = interaction_values[interaction_values != np.array(None)]
  assert x.shape[0] == y.shape[0]
  assert x.ndim == 1
  assert y.ndim == 1
  return x, y

In [ ]:
#@title Interaction fraction within player function
def interaction_fraction_within_player(
    *,
    first_target_x: np.ndarray,
    first_target_y: np.ndarray,
    second_target_x: np.ndarray,
    second_target_y: np.ndarray,
    reference_x: Optional[np.ndarray] = None,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
  """Get interaction_fractions of which target agent the evaluated agent interacts with.

  In order to get the interaction_fraction of who the agent tends to interact
  with we first
  need to bin the data into chunks of the x axis (learner steps).

  Args:
    first_target_x: Agent learner steps of the first target agent eval.
    first_target_y: Interactions with the first target agent.
    second_target_x: Agent learner steps of the second target agent eval.
    second_target_y: Interactions with the second target agent.
    reference_x: Reference x axis values for the interactions. If None it will
    be created.

  Returns:
    Tuple of numpy arrays with the x (learner steps) and y values
    of the interactions (interaction_fraction between players).
  """
  new_x = []
  new_y = []
  if reference_x is None:
    reference_x = np.unique(np.round(first_target_x / CATCH))
  for x_bin in reference_x:
    new_x.append(x_bin * CATCH)
    first_target_x_indices = np.where(np.round(first_target_x / CATCH) == x_bin)
    first_target_y_vals = np.sum(first_target_y[first_target_x_indices])
    second_target_x_indices = np.where(
        np.round(second_target_x / CATCH) == x_bin
    )
    second_target_y_vals = np.sum(second_target_y[second_target_x_indices])

    if (first_target_y_vals + second_target_y_vals) == 0:
      interaction_fraction = 0.5
    else:
      interaction_fraction = float(
          first_target_y_vals / (first_target_y_vals + second_target_y_vals)
      )
    new_y.append(interaction_fraction)
  new_x = np.array(new_x)
  new_y = np.squeeze(np.array(new_y))
  assert new_x.shape == new_y.shape
  assert new_x.ndim == 1
  assert new_y.ndim == 1
  assert reference_x.ndim == 1
  return new_x, new_y, reference_x

In [ ]:
#@title Overview plot function
def overview_plot(
    *,
    x: np.ndarray,
    y_differences: List[float],
    y_errors: List[float],
    legend_strings: str,
    ylims: Tuple[float],
    smooth_size: int = 5,
) -> List[float]:
  """Plot overviews of the group bias.

  Args:
    x: Values for X axis.
    y_differences: Differences in the approach behavior for y axis.
    y_errors: Error for the y axis.
    legend_strings: Content for legend (one string per run).
    ylims: Y axis limits.
    smooth_size: How much to smooth the data for plotting.

  Returns:
    List of group biases (one per run).
  """
  size = (5, 3)
  setting_colors = [
      '#6b6b73',
      '#f58231',
      '#dcbeff',
      '#800000',
      '#ffe119',
      '#000075',
      '#a9a9a9',
      '#000000',
  ]
  bias_means = []
  for i in range(len(y_differences)):
    bias_means.append(np.mean(y_differences[i]))

  plt.figure(facecolor='white', figsize=size)
  plt.bar(range(len(y_differences)), np.array(bias_means), color=setting_colors)
  plt.ylim(ylims)
  plt.xticks(range(len(y_differences)), legend_strings)
  plt.xlabel('Setting')
  plt.ylabel('Mean of group bias over time')

  plt.savefig('overview_bars.svg', dpi=200)

  plt.figure(figsize=size)
  for i in range(len(y_differences)):
    y = smooth(y_differences[i], smooth_size=smooth_size)
    plt.plot(x, y, c=setting_colors[i])
    plt.fill_between(
        x,
        y - y_errors[i],
        y + y_errors[i],
        alpha=0.1,
        label='_nolegend_',
        color=setting_colors[i],
    )

  plt.hlines(0, 0, max(x), 'black')
  plt.ylabel('In group bias')
  plt.xlabel('Training steps')
  plt.xticks([0, max(x)], [0, '1.5e9'])
  plt.xticks(
      [0, max(x) / 3, (max(x) / 3) * 2, max(x)], ['0', '0.5e9', '1e9', '1.5e9']
  )

  plt.ylim(-0.3, 1.1)
  plt.savefig('overview.svg', dpi=200)
  plt.show()

  return bias_means

In [ ]:
#@title Individuation analyses function
def individuation_analyses(
    *,
    freeze_levels: List[int],
    red_individuation: Tuple[List[float], ...],
    blue_individuation: Tuple[List[float], ...],
    bias: List[np.ndarray],
    label: str,
) -> Tuple[np.ndarray]:
  """Plot and analyse the individuation experiment.

  Args:
    freeze_levels: How negative the interaction is with the least desirable
    agent (one value per run).
    red_individuation: Individuation evaluation results of red agents.
    blue_individuation: Individuation evaluation results of blue agents.
    bias: Group bias evaluation results.
    label: Label of experiment.

  Returns:
    Tuple containing the difference of individuation and group bias.
  """
  size = (3, 2)
  freeze_levels_str = [str(x) for x in freeze_levels]
  number_of_runs = len(freeze_levels_str)
  assert len(red_individuation[1]) == len(blue_individuation[1])
  assert len(red_individuation[1]) == number_of_runs
  assert len(blue_individuation[1]) == len(bias)
  avoidance_of_less_rewarding_agent_ingroup_y = np.array([
      1 - (red_individuation[0][x] + blue_individuation[1][x] / 2)
      for x in range(number_of_runs)
  ])
  avoidance_of_less_rewarding_agent_outgroup_y = np.array([
      1 - (red_individuation[1][x] + blue_individuation[0][x] / 2)
      for x in range(number_of_runs)
  ])
  avoidance_of_less_rewarding_agent_ingroup_error = np.array([
      (red_individuation[2][x] + blue_individuation[3][x] / 2)
      for x in range(number_of_runs)
  ])
  avoidance_of_less_rewarding_agent_outgroup_error = np.array([
      (red_individuation[3][x] + blue_individuation[2][x] / 2)
      for x in range(number_of_runs)
  ])

  plt.figure(facecolor='white', figsize=size)
  plt.plot(
      freeze_levels, avoidance_of_less_rewarding_agent_ingroup_y, color='teal'
  )
  plt.fill_between(
      freeze_levels,
      avoidance_of_less_rewarding_agent_ingroup_y
      - avoidance_of_less_rewarding_agent_ingroup_error,
      avoidance_of_less_rewarding_agent_ingroup_y
      + avoidance_of_less_rewarding_agent_ingroup_error,
      alpha=0.2,
      label='_nolegend_',
      color='teal',
  )
  plt.plot(
      freeze_levels,
      avoidance_of_less_rewarding_agent_outgroup_y,
      color='magenta',
  )
  plt.fill_between(
      freeze_levels,
      avoidance_of_less_rewarding_agent_outgroup_y
      - avoidance_of_less_rewarding_agent_outgroup_error,
      avoidance_of_less_rewarding_agent_outgroup_y
      + avoidance_of_less_rewarding_agent_outgroup_error,
      alpha=0.2,
      label='_nolegend_',
      color='magenta',
  )
  plt.legend(['In-group', 'Out-group'])
  plt.xticks(freeze_levels, freeze_levels_str)
  plt.xlabel('Freeze time')
  plt.ylabel('Individuation')
  plt.title(label)
  plt.ylim(0, 1)
  plt.savefig('discrimination.svg', dpi=200)

  plt.figure(facecolor='white', figsize=size)
  plt.bar(
      range(number_of_runs),
      avoidance_of_less_rewarding_agent_ingroup_y
      - avoidance_of_less_rewarding_agent_outgroup_y,
      color='k',
  )
  plt.xticks(range(number_of_runs), freeze_levels_str)
  plt.xlabel('freeze time')
  plt.ylim(0, 0.4)
  plt.ylabel('Difference in individuation')

  individuation_difference = np.array(
      avoidance_of_less_rewarding_agent_ingroup_y
      - avoidance_of_less_rewarding_agent_outgroup_y
  )
  plt.savefig('discrimination_difference.svg', dpi=200)

  plt.figure(facecolor='white', figsize=size)
  group_bias = np.array([np.mean(data) for data in bias])
  grey_values = np.linspace(0.2, 1, num=number_of_runs)
  for i in range(number_of_runs):
    plt.scatter(
        individuation_difference[i],
        group_bias[i],
        marker='.',
        c='black',
        s=500,
        alpha=grey_values[i],
    )

  plt.ylabel('In group bias')
  plt.xlabel('Difference in individuation hard pixel')
  plt.xlim(0, 0.35)
  plt.ylim(0.2, 0.35)
  plt.xticks([0, 0.1, 0.2, 0.3])
  plt.yticks([0.2, 0.25, 0.3, 0.35])

  correlation = scipy.stats.pearsonr(group_bias, individuation_difference)
  print(
      'Group bias and discrimination difference: r',
      np.round(correlation[0], decimals=2),
      'p=',
      np.round(correlation[1], decimals=3),
  )
  b, a = np.polyfit(individuation_difference, group_bias, deg=1)
  xseq = np.linspace(
      min(individuation_difference), max(individuation_difference), num=100
  )
  # Plot regression line
  plt.plot(xseq, a + b * xseq, color='grey', lw=2.5)
  plt.savefig('discrimination_bias_correlation.svg', dpi=200)
  return individuation_difference, group_bias

In [ ]:
#@title Smoothing function
def smooth(data: np.ndarray, *, smooth_size: int) -> np.ndarray:
  """A common smoothing function.

  Args:
    data: Array to be smoothed.
    smooth_size: How much to smooth the data.

  Returns:
    Smoothed array.
  """
  if smooth_size == -1:
    out = data
  else:
    alpha = 2 / (smooth_size + 1.0)
    alpha_rev = 1 - alpha
    n = data.shape[0]

    pows = alpha_rev ** (np.arange(n + 1))

    scale_arr = 1 / pows[:-1]
    offset = data[0] * pows[1:]
    pw0 = alpha * alpha_rev ** (n - 1)

    mult = data * pw0 * scale_arr
    cumsums = mult.cumsum()
    out = offset + cumsums * scale_arr[::-1]
  return out

Figure 1

In [ ]:
df_list = load_dataframes(
    file_label='fig1', number_of_files=5, downsample=DOWNSAMPLE
)
x, y_differences, y_errors, _, _ = plot_metric_style(
    dataframes=df_list,
    eval_name='two_colors',
    first_choice='red',
    second_choice='blue',
    number_of_timebins=NUMBER_OF_TIMEBINS,
    number_of_agents_to_eval=6,
    titles=LEGEND,
)
bias_sums_fig1 = overview_plot(
    x=x,
    y_differences=y_differences,
    y_errors=y_errors,
    legend_strings=LEGEND,
    ylims=YLIM,
)

Fig 2 A & 5 A

In [ ]:
df_list_correlated_colors = load_dataframes(
    file_label='fig2_correlated_colors',
    number_of_files=5,
    downsample=DOWNSAMPLE
    )
x, ydiffs_correlated_colors, y_errors_correlated_colors, _, _ = (
    plot_metric_style(
        dataframes=df_list_correlated_colors,
        eval_name='two_colors',
        first_choice='red',
        second_choice='blue',
        number_of_timebins=NUMBER_OF_TIMEBINS,
        number_of_agents_to_eval=6,
        titles=LEGEND
        )
)
bias_sums_close = overview_plot(
    x=x,
    y_differences=ydiffs_correlated_colors,
    y_errors=y_errors_correlated_colors,
    legend_strings=LEGEND,
    ylims=YLIM
    )

In [ ]:
df_list_very_correlated_colors = load_dataframes(
    file_label='fig2_very_correlated_colors',
    number_of_files=5,
    downsample=DOWNSAMPLE,
)
x, ydiffs_list_very_correlated_colors, y_errors_very_correlated_colors, _, _ = (
    plot_metric_style(
        dataframes=df_list_very_correlated_colors,
        eval_name='two_colors',
        first_choice='red',
        second_choice='blue',
        number_of_timebins=NUMBER_OF_TIMEBINS,
        number_of_agents_to_eval=6,
        titles=LEGEND,
    )
)
bias_sums_very_close = overview_plot(
    x=x,
    y_differences=ydiffs_list_very_correlated_colors,
    y_errors=y_errors_very_correlated_colors,
    legend_strings=LEGEND,
    ylims=YLIM,
)

In [ ]:

df_list_one_resource = load_dataframes(
    file_label='fig5_one_resource', number_of_files=5, downsample=DOWNSAMPLE
)
x, ydiffs_one_color, y_errors_one_resource, _, _ = plot_metric_style(
    dataframes=df_list_one_resource,
    eval_name='two_colors',
    first_choice='red',
    second_choice='blue',
    number_of_timebins=NUMBER_OF_TIMEBINS,
    number_of_agents_to_eval=6,
    titles=LEGEND,
)
bias_sums_one_color = overview_plot(
    x=x,
    y_differences=ydiffs_one_color,
    y_errors=y_errors_one_resource,
    legend_strings=LEGEND,
    ylims=YLIM,
)


In [ ]:
setting_colors = [
    '#6b6b73',
    '#f58231',
    '#dcbeff',
    '#800000',
    '#ffe119',
    '#000075',
    '#a9a9a9',
    '#000000',
]
plt.subplots(1, 1, facecolor='white', figsize=(4, 4))

biases = np.stack((bias_sums_one_color, bias_sums_fig1[:5]))

plt.subplot(1, 1, 1)
for i in range(5):
  plt.plot(biases[:, i], 'o-', c=setting_colors[i])
plt.legend(
    ['8:0|8:0', '8:0|7:1', '8:0|6:2', '8:0|5:3', '8:0|4:4'],
    bbox_to_anchor=(0.85, 0.75),
)

plt.xticks([0, 1], ['one color', 'three colors'])
plt.xlabel('Number of resources')
plt.ylabel('In group bias')
plt.ylim(-0, 1)
plt.xlim(-0.2, 1.2)

plt.savefig('one_color.svg', dpi=200)

biases = np.stack((bias_sums_fig1[:5], bias_sums_close, bias_sums_very_close))
plt.subplots(1, 1, facecolor='white', figsize=(4, 4))
plt.subplot(1, 1, 1)
for i in range(5):
  plt.plot(biases[:, i], 'o-', c=setting_colors[i])
plt.xticks([0, 1, 2], ['300', '200', '100'])
plt.xlabel('RGB distance')
plt.ylim(0, 1)
plt.xlim(-0.3, 2.3)
plt.ylabel('In group bias')

plt.savefig('similar_colors.svg', dpi=200)


In [ ]:
df_list_walled = load_dataframes(
    file_label='fig2_walled', number_of_files=5, downsample=DOWNSAMPLE
)
x, ydiffs_walled, y_errors_walled, _, _ = plot_metric_style(
    dataframes=df_list_walled,
    eval_name='two_colors',
    first_choice='red',
    second_choice='blue',
    number_of_timebins=NUMBER_OF_TIMEBINS,
    number_of_agents_to_eval=6,
    titles=LEGEND,
)
bias_sums_walled = overview_plot(
    x=x,
    y_differences=ydiffs_walled,
    y_errors=y_errors_walled,
    legend_strings=LEGEND,
    ylims=YLIM,
)


In [ ]:

df_list_celled = load_dataframes(
    file_label='fig2_celled', number_of_files=5, downsample=DOWNSAMPLE
)
x, ydiffs_celled, y_errors_celled, _, _ = plot_metric_style(
    dataframes=df_list_celled,
    eval_name='two_colors',
    first_choice='red',
    second_choice='blue',
    number_of_timebins=NUMBER_OF_TIMEBINS,
    number_of_agents_to_eval=6,
    titles=LEGEND,
)
bias_sums_celled = overview_plot(
    x=x,
    y_differences=ydiffs_celled,
    y_errors=y_errors_celled,
    legend_strings=LEGEND,
    ylims=YLIM,
)


Figure 4

In [ ]:

freeze_levels = [16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64]

df_list_individuation = load_dataframes(
    file_label='fig4_brighter_pixel', number_of_files=13, downsample=DOWNSAMPLE
)

_, ydiffs_individuation, _, _, _ = plot_metric_style(
    dataframes=df_list_individuation,
    eval_name='two_colors',
    first_choice='red',
    second_choice='blue',
    number_of_timebins=NUMBER_OF_TIMEBINS,
    number_of_agents_to_eval=4,
    titles=freeze_levels,
)
_, _, _, discplot_individuation_red, _ = plot_metric_style(
    dataframes=df_list_individuation,
    eval_name='red_bad_v_red_new',
    first_choice='red_bad',
    second_choice='red_new',
    number_of_timebins=NUMBER_OF_TIMEBINS,
    number_of_agents_to_eval=4,
    titles=freeze_levels,
)
_, _, _, discplot_individuation_blue, _ = plot_metric_style(
    dataframes=df_list_individuation,
    eval_name='blue_bad_v_blue_new',
    first_choice='blue_bad',
    second_choice='blue_new',
    number_of_timebins=NUMBER_OF_TIMEBINS,
    number_of_agents_to_eval=4,
    titles=freeze_levels,
)

individuation_difference_easy, group_bias_easy = individuation_analyses(
    freeze_levels=freeze_levels,
    red_individuation=discplot_individuation_red,
    blue_individuation=discplot_individuation_blue,
    bias=ydiffs_individuation,
    label='Brighter pixel (easier individuation)',
)


In [ ]:
df_list_individuation_duller = load_dataframes(
    file_label='fig4_duller_pixel', number_of_files=13, downsample=DOWNSAMPLE
)

_, ydiffs_individuation_duller, _, _, _ = plot_metric_style(
    dataframes=df_list_individuation_duller,
    eval_name='two_colors',
    first_choice='red',
    second_choice='blue',
    number_of_timebins=NUMBER_OF_TIMEBINS,
    number_of_agents_to_eval=4,
    titles=freeze_levels,
)
_, _, _, discplot_individuation_duller_red, _ = plot_metric_style(
    dataframes=df_list_individuation_duller,
    eval_name='red_bad_v_red_new',
    first_choice='red_bad',
    second_choice='red_new',
    number_of_timebins=NUMBER_OF_TIMEBINS,
    number_of_agents_to_eval=4,
    titles=freeze_levels,
)
_, _, _, discplot_individuation_duller_blue, _ = plot_metric_style(
    dataframes=df_list_individuation_duller,
    eval_name='blue_bad_v_blue_new',
    first_choice='blue_bad',
    second_choice='blue_new',
    number_of_timebins=NUMBER_OF_TIMEBINS,
    number_of_agents_to_eval=4,
    titles=freeze_levels,
)

individuation_difference_middle, group_bias_middle = individuation_analyses(
    freeze_levels=freeze_levels,
    red_individuation=discplot_individuation_duller_red,
    blue_individuation=discplot_individuation_duller_blue,
    bias=ydiffs_individuation_duller,
    label='Duller pixel (harder individuation)',
)


Copyright 2024 DeepMind Technologies Limited.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.